In [1]:
!pip install validator-collection
!pip install kaggle

In [2]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import os
import urllib.request, urllib.error
from validator_collection import validators, checkers
import re

In [3]:
project_path = '/home/mgarcia/Documentos/ironhack/ironhack-projects/pandas-project/your-code/' 

In [4]:
api = KaggleApi()
api.authenticate()


#In case you need to download the file, use this:

#api.dataset_download_files('teajay/global-shark-attacks')
#file = 'global-shark-attacks.zip'
#with zipfile.ZipFile(project_path + file, 'r') as zip_ref:
#    zip_ref.extractall(project_path)

In [5]:
raw = pd.read_csv(project_path + 'attacks.csv', encoding = 'latin_1')
raw.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Hypothesis:
1. Most attacks were unprovoked
2. Most victims were males between 20 and 40 years
3. Most victims were surfing

## Cleaning empty rows

In [6]:
print(raw.shape)
raw.isnull().sum()

(25723, 24)


Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [7]:
# if case number is null, maybe all the row is null. Let's check
raw_clean=raw[~raw['Case Number'].isnull()]
print(raw_clean.shape)
raw_clean.isnull().sum()

# indeed :D

(8702, 24)


Case Number                  0
Date                      2401
Year                      2403
Type                      2405
Country                   2451
Area                      2856
Location                  2941
Activity                  2945
Name                      2611
Sex                       2966
Age                       5231
Injury                    2429
Fatal (Y/N)               2940
Time                      5754
Species                   5238
Investigator or Source    2418
pdf                       2401
href formula              2402
href                      2401
Case Number.1             2401
Case Number.2             2401
original order            2394
Unnamed: 22               8701
Unnamed: 23               8700
dtype: int64

In [8]:
# Also, seems like null dates mean empty row
raw_clean=raw[~raw['Date'].isnull()]
print(raw_clean.shape)
raw_clean.isnull().sum()

# confirm

(6302, 24)


Case Number                  1
Date                         0
Year                         2
Type                         4
Country                     50
Area                       455
Location                   540
Activity                   544
Name                       210
Sex                        565
Age                       2831
Injury                      28
Fatal (Y/N)                539
Time                      3354
Species                   2838
Investigator or Source      17
pdf                          0
href formula                 1
href                         0
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               6301
Unnamed: 23               6300
dtype: int64

In [9]:
# The dataset has reduce a good amount of data, so now it will be easier to inspect it
raw_clean=raw_clean.reset_index(drop=True)
raw_clean.to_csv('attacks_clean.csv', ';', quotechar = '"')

 ## Dropping unnecesary columns


In [10]:
# two last columns are empty so we can drop them
#columns = raw.columns
raw_clean = raw_clean.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

# As we're taking Case Number as reference, we don't need the duplicated Case Number columns
raw_clean = raw_clean.drop(columns=['Case Number.1', 'Case Number.2'])

# original order isn't useful for our purposes
raw_clean = raw_clean.drop(columns=['original order'])

In [11]:
# href and href formula looks pretty much the same (href formula has one extra null)
(raw_clean['href formula'] == raw_clean['href']).value_counts()

True     6242
False      60
dtype: int64

In [12]:
#The code below is quite heavy, I wouldn't run it unless I'm sure I have time. 
#The results were:
#requests_failed =0
#invalid_url = 73

#requests_failed = 0
#invalid_url = 0
#for url in raw_clean['href']:
#    if checkers.is_url(url) == True:
#        try:
#            resp = urllib.request.urlopen(url).getcode()
#        except urllib.error.HTTPError as e:
#            if e.code == 404:
#                requests_failed = requests_failed + 1
#    else:
#        invalid_url = invalid_url + 1

#print(requests_failed)
#print(invalid_url)

In [13]:
#So we can drop one of the columns without losing a lot of information as they match at 99%
raw_clean = raw_clean.drop(columns=['href formula'])

In [14]:
# pdf is just the name of the pdf file that stores the data, as long as we can donwload it. 

raw_clean = raw_clean.drop(columns=['pdf'])

## Deleting duplicates

In [15]:
# first check: full duplicate rows
print(sum(raw_clean.duplicated()))
raw_clean = raw_clean.drop_duplicates()

0


In [16]:
# second check: case numbers
print(raw_clean.duplicated(subset='Case Number').sum())

pd.concat(g for _, g in raw_clean.groupby('Case Number') if len(g) > 1)

#Can't drop this like that. Let's check date + country + area to verify duplicity

raw_clean = raw_clean.drop_duplicates(subset=['Case Number', 'Date', 'Country', 'Area'], keep = False)

16


In [17]:
# third check: href. If it points to the same report, it probaby be the same case
print(raw_clean.duplicated(subset='href').sum())

pd.concat(g for _, g in raw_clean.groupby("href") if len(g) > 1)

raw_clean = raw_clean.drop_duplicates(subset=['href', 'Date', 'Country', 'Area'], keep = False)


15


In [18]:
# As we're finished reshaping the dataframe, it's a good moment to reset index
raw_clean=raw_clean.reset_index(drop=True)

## Formatting data

In [19]:
raw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6276 entries, 0 to 6275
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             6275 non-null   object 
 1   Date                    6276 non-null   object 
 2   Year                    6274 non-null   float64
 3   Type                    6272 non-null   object 
 4   Country                 6226 non-null   object 
 5   Area                    5821 non-null   object 
 6   Location                5736 non-null   object 
 7   Activity                5734 non-null   object 
 8   Name                    6068 non-null   object 
 9   Sex                     5711 non-null   object 
 10  Age                     3457 non-null   object 
 11  Injury                  6248 non-null   object 
 12  Fatal (Y/N)             5739 non-null   object 
 13  Time                    2938 non-null   object 
 14  Species                 3452 non-null   

In [20]:
# Case number should be string
raw_clean['Case Number'] = raw_clean['Case Number'].str.strip()
raw_clean['Case Number'] = raw_clean['Case Number'].astype(str)

In [45]:
# Date should be date, but we have some "Reported"
DateRegex = re.compile(r'\d{2}-\w{3}-\d{4}')

        
#print(~[DateRegex.search(date) for date in raw_clean['Date']])

raw_clean['Date'] = raw_clean['Date'].replace('Reported', '', regex=True).replace('Before', '', regex=True).replace('No date', '', regex=True).replace('Between', '', regex=True).replace('.', '-', regex=True).str.strip()

count = 0
for date in raw_clean['Date']:
    if not DateRegex.search(date):
        count = count +1
        #print (date)
print (count)

# This too complicated to fix in a reasonable amount of time


6263


In [22]:
# as Date is a mess, at least we can fix Year so we can have something to work with in the future

raw_clean['Year'] = raw_clean['Year'].fillna(0)


print(raw_clean[raw_clean['Year'] == 0].count()['Year'])
raw_clean['Date'].loc[raw_clean['Year']==0]

# If Date has a year, we can use it to fill the Year gap
for i in range(len(raw_clean['Year'])):
    if raw_clean['Year'][i] == 0:
        if len(re.findall('\d{4}', raw_clean['Date'][i])) > 0:
            raw_clean['Year'][i] = re.findall('\d{4}', raw_clean['Date'][i])[0]

127


/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [23]:
print(raw_clean[raw_clean['Year'] == 0].count()['Year'])
raw_clean['Date'].loc[raw_clean['Year']==0]

# The remaining 13 zeros are not important so we can drop them without losing a lot of info
raw_clean=raw_clean[raw_clean.Year!=0]

# cast to integer
raw_clean['Year'] = [int(round(y)) for y in raw_clean['Year']]

13


In [24]:
# Type has very specific categories but some easy to fix mispellings

print(raw_clean['Type'].value_counts())

raw_clean['Type']=raw_clean['Type'].replace('Boat','Boating')
raw_clean['Type']=raw_clean['Type'].replace('Boatomg','Boating')

raw_clean['Type'].value_counts()

Unprovoked      4566
Provoked         572
Invalid          545
Sea Disaster     235
Boating          201
Boat             137
Questionable       2
Boatomg            1
Name: Type, dtype: int64


Unprovoked      4566
Provoked         572
Invalid          545
Boating          339
Sea Disaster     235
Questionable       2
Name: Type, dtype: int64

In [25]:
# Sex column has spaces and this may happen in other cases so let's fix them

raw_clean.rename(columns=lambda x: x.strip(), inplace=True)

print(raw_clean['Sex'].value_counts())

# N is right next to M in most keyborads so I can guess is a typo
raw_clean['Sex']=raw_clean['Sex'].replace('N','M', inplace=True)

# Second M category has an space
raw_clean['Sex']=raw_clean['Sex'].replace('M ','M', inplace=True)

print(raw_clean['Sex'].value_counts())

M      5060
F       632
M         2
N         2
lli       1
.         1
Name: Sex, dtype: int64
Series([], Name: Sex, dtype: int64)


In [55]:
# Age has a lot of NaN values. To avoid future errors, we're filling it with Zero values
print(raw_clean['Age'].value_counts())
print(raw_clean['Age'].isnull().value_counts())

raw_clean['Age'] = raw_clean['Age'].fillna(0)

# cast to string to apply transformations later
raw_clean['Age'] = [str(x).strip() for x in raw_clean['Age']]


# We can also do some stripping in case it hepls
raw_clean['Age'] = raw_clean['Age'].str.strip()

raw_clean['Age'] = raw_clean['Age'].str.replace('&',',', regex=True)
raw_clean['Age'] = raw_clean['Age'].str.replace('½',',', regex=True)

# we still have a few that contains "or" and "to"
raw_clean[raw_clean['Age'].str.contains('to')]
raw_clean[raw_clean['Age'].str.contains('or')]

print(raw_clean['Age'].value_counts())

0                2806
17                154
18                151
19                142
20                141
                 ... 
2,                  1
46 , 34             1
25 to 35            1
18 to 22            1
2 to 3 months       1
Name: Age, Length: 138, dtype: int64
False    6263
Name: Age, dtype: int64
0                2806
17                154
18                151
19                142
20                141
                 ... 
2,                  1
46 , 34             1
25 to 35            1
18 to 22            1
2 to 3 months       1
Name: Age, Length: 138, dtype: int64


In [54]:
# delete the 'or' and pick the first age, it's usually close to truth and that way we're allowed to group by
#for i in range(len(raw_clean['Age'])):
#    if 'or' in raw_clean['Age'][i]:
#        print(raw_clean['Age'][i])
 if :aw_clean[raw_clean['Age'].str.contains('to')]


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,href
2754,1987.12.13,-----------,1987,Invalid,AUSTRALIA,New South Wales,Shoalhaven,NaN,female,None,16 to 18,"Remains recovered from 4 m, 420-kg, caught 13-...",NaN,NaN,Shark involvement prior to death was not confi...,"Sunday Mail (QLD), 1/31/1988, p.2",http://sharkattackfile.net/spreadsheets/pdf_di...
3369,1970.12.13,-----------,1970,Unprovoked,MOZAMBIQUE,Inhambe Province,"Inhambe Bay Estuary, 10 to 12 miles inland fro...",Fishing for prawns,black male,None,18 to 22,"FATAL, decapitated and arm severed",Y,NaN,NaN,"GSAF; H.D. Baldridge, p.22",http://sharkattackfile.net/spreadsheets/pdf_di...
3640,1965.00.00.c,------------,1965,Unprovoked,EGYPT,NaN,South of Hurghada,"Spearfishing, but standing in the water",male,None,25 to 35,Ankle and thigh bitten; shark made three strikes,N,NaN,"1.5 m to 2.1 m [5' to 7'] shark, possibly a ma...",Captain T. Falcon-Barker,http://sharkattackfile.net/spreadsheets/pdf_di...
4205,1957.00.00.i,----,1957,Invalid,CUBA,Havana Province,Cojimar,NaN,an infant,None,2 to 3 months,"FATAL, but was it an accident or infanticide?",NaN,NaN,3.7 m [12'] shark,"F. Poli, p.23-24",http://sharkattackfile.net/spreadsheets/pdf_di...
